# Day0 — Setup & Preprocessing Pipeline

목표:
- 데이터 구조(Time 반복) 확인 및 run 분리 기준 확정
- Prophet 입력용 연속 시간축(`abs_time_ms`) 생성 방식 확정
- 단위/스케일 가정(임시) 정리 후 전처리 코드로 재현 가능하게 고정
- processed 데이터 생성 및 기본 검증


## 데이터 정의
상세 정의는 `tables/Day0/Day0_data_dictionary.csv` 참고.

- `Time`: run 내 상대시간(ms), run마다 0부터 반복
- `run_id`: `Time`이 감소하는 지점(예: 4950→0)을 run 경계로 생성
- `abs_time_ms`: `run_id * run_period_ms + Time` (연속 시간축)
- 스케일(임시):
  - `Temp**`, `Press**`, `FB_Torque`: 값 분포상 ×10 형태 가능 → **÷10 적용(가정)**
  - `FB_Rpm`: **그대로 사용(가정)**
  - `Vib_X/Y/Z`: 단위 환산 보류, 상대 지표로 사용(대표값 생성)

※ 단위/스케일은 추후 변경 가능(전처리 파이프라인은 유지).


## 전처리 파이프라인
`src/data_make.py` 기준 처리 흐름:

raw CSV 로딩 → 컬럼 정리 → `run_id` 생성 → 정렬(run_id, Time) → `abs_time_ms` 생성  
→ 스케일 가정 적용 → 대표 센서(feature) 생성 → processed 저장


In [ ]:
from pathlib import Path
import pandas as pd

from src.data_make import make_processed_dataset, save_processed_dataset, DataConfig

INPUT_CSV = Path("../../data/raw/Basic_Model_20251015_15.csv")
OUTPUT_CSV = Path("../../data/processed/processed_v1_run_abs_features.csv")

INPUT_CSV, OUTPUT_CSV

## 전처리 실행 (processed 생성)

In [ ]:
cfg = DataConfig(
    enable_scale=True,
    apply_rpm_scale=False,  # Day1에서 결정 전까지 RPM 변환 보류
)

df = make_processed_dataset(INPUT_CSV, cfg=cfg)
save_processed_dataset(df, OUTPUT_CSV)

print(f"[OK] saved: {OUTPUT_CSV} (rows={len(df)}, cols={len(df.columns)})")
df.head(3)

## 결과 검증 1) run 구조 확인
- run 개수(`run_id` unique)
- run당 샘플 수(평균/최소/최대)


In [ ]:
runs = df["run_id"].nunique()
run_size_desc = df.groupby("run_id").size().describe()

runs, run_size_desc

## 결과 검증 2) 스케일/범위 sanity check
- Torque/Temp/Press는 ÷10 적용 상태에서 범위 확인
- RPM은 그대로 사용 상태(max≈145 수준) 확인

In [ ]:
cols = ["FB_Torque", "FB_Rpm"]
if "Temp_mean" in df.columns:
    cols.append("Temp_mean")
if "Press_mean" in df.columns:
    cols.append("Press_mean")

df[cols].describe().loc[["min", "max", "mean"]]

## 결과 검증 3) 결측치 체크

In [ ]:
df.isna().sum().sort_values(ascending=False).head(15)

## Day0 결론
- run 분리 및 연속 시간축(`abs_time_ms`) 생성 파이프라인 확정
- 스케일 가정(Temp/Press/Torque ÷10, RPM 그대로)을 적용한 processed 데이터 생성 완료
- Day1에서는 전체 시간축 기준 주요 센서 그래프 및 요약 테이블 생성으로 넘어간다